# B-state energy levels 
Calculating the energy levels and eigenstates of the B-state Hamiltonian

Import the relevant packages and Hamiltonians

In [63]:
import numpy as np
import pickle
import sys
import sympy

sys.path.append('../B-state-classes-and-functions/')

from classes import CoupledBasisState, UncoupledBasisState, State
from functions import matrix_to_states, vector_to_state, find_state_idx_from_state

with open("B_hamiltonians_symbolic_coupled_P.py",'rb') as f:
    hamiltonians = pickle.load(f)

def ni_range(x0, x1, dx=1):
    # sanity check arguments
    if dx==0:
        raise ValueError("invalid parameters: dx==0")
    if x0>x1 and dx>=0:
        raise ValueError("invalid parameters: x0>x1 and dx>=0")
    if x0<x1 and dx<=0:
        raise ValueError("invalid parameters: x0<x1 and dx<=0")
        
    # generate range list
    range_list = []
    x = x0
    while x < x1:
        range_list.append(x)
        x += dx
    return range_list

#Generate quantum numbers
Jmin = 1
Jmax = 6 # max J value in Hamiltonian
#Jmax = 6
I_Tl = 1/2             # I1 in Ramsey's notation
I_F  = 1/2             # I2 in Ramsey's notation


Omegas = [-1, 1]
# QN = [UncoupledBasisState(J,mJ,I_Tl,m1,I_F,m2, Omega)
#       for J  in ni_range(Jmin, Jmax+1)
#       for Omega in Omegas
#       for mJ in ni_range(-J,J+1)
#       for m1 in ni_range(-I_Tl,I_Tl+1)
#       for m2 in ni_range(-I_F,I_F+1)
#      ]

Ps = [-1,1]
QN = [CoupledBasisState(F,0,F1,J,I_F,I_Tl, P = P)
      for J  in ni_range(Jmin, Jmax+1)
      for F1 in ni_range(np.abs(J-I_F),J+I_F+1)
      for F in ni_range(np.abs(F1-I_Tl),F1+I_Tl+1)
      for P in Ps
     ]

In [64]:
variables = [
    *sympy.symbols('Brot Drot H_const'),
    *sympy.symbols('h1_Tl h1_F'),
    sympy.symbols('q'),
    sympy.symbols('c_Tl'),
    sympy.symbols('c1p_Tl')
]

lambdified_hamiltonians = {
    H_name : sympy.lambdify(variables, H_matrix)
    for H_name, H_matrix in hamiltonians.items()
}

Insert the numerical constants:

In [65]:
#Constants in MHz
Brot = 6687.879e6
Drot = 0.010869e6
H_const = -8.1e-2
h1_Tl = 28789e6
h1_F = 861e6
q = 2.423e6
c_Tl = -7.83e6
c1p_Tl = 11.17e6

H = {
    H_name : H_fn(
        Brot, Drot, H_const,
        h1_Tl, h1_F,
        q,
        c_Tl,
        c1p_Tl
    )
    for H_name, H_fn in lambdified_hamiltonians.items()
}



Check that Hamiltonian is hermitian and diagonalize the Hamiltonian

In [66]:
Hff = H["Hrot"]+H["H_mhf_Tl"]+H["H_mhf_F"]+H["H_c_Tl"]+H["H_cp1_Tl"]+H["H_LD"]
print(np.allclose(Hff, Hff.conj().T))
D, V = np.linalg.eigh(Hff)

True


In [67]:
states = matrix_to_states(V,QN)

In [68]:
for state in states[:10]:
    print("")
    state.remove_small_components(tol = 0.01).print_state(tol = 0.01)


+1.0000+0.0000j x |J = 1, F1 = 1/2, F = 0, mF = 0, I1 = 1/2, I2 = 1/2, Omega = 1, P = 1>

+1.0000+0.0000j x |J = 1, F1 = 1/2, F = 0, mF = 0, I1 = 1/2, I2 = 1/2, Omega = 1, P = -1>

-0.9996+0.0000j x |J = 1, F1 = 1/2, F = 1, mF = 0, I1 = 1/2, I2 = 1/2, Omega = 1, P = 1>
-0.0200+0.0000j x |J = 1, F1 = 3/2, F = 1, mF = 0, I1 = 1/2, I2 = 1/2, Omega = 1, P = 1>
-0.0178+0.0000j x |J = 2, F1 = 3/2, F = 1, mF = 0, I1 = 1/2, I2 = 1/2, Omega = 1, P = 1>

+0.9996+0.0000j x |J = 1, F1 = 1/2, F = 1, mF = 0, I1 = 1/2, I2 = 1/2, Omega = 1, P = -1>
+0.0201+0.0000j x |J = 1, F1 = 3/2, F = 1, mF = 0, I1 = 1/2, I2 = 1/2, Omega = 1, P = -1>
+0.0178+0.0000j x |J = 2, F1 = 3/2, F = 1, mF = 0, I1 = 1/2, I2 = 1/2, Omega = 1, P = -1>

-0.0264+0.0000j x |J = 1, F1 = 1/2, F = 1, mF = 0, I1 = 1/2, I2 = 1/2, Omega = 1, P = -1>
+0.8518+0.0000j x |J = 1, F1 = 3/2, F = 1, mF = 0, I1 = 1/2, I2 = 1/2, Omega = 1, P = -1>
+0.5233+0.0000j x |J = 2, F1 = 3/2, F = 1, mF = 0, I1 = 1/2, I2 = 1/2, Omega = 1, P = -1>

+0.0264+

In [69]:
D

array([-1.44946348e+09, -1.43344748e+09, -8.85785272e+08, -8.69797942e+08,
        1.26311663e+10,  1.26488710e+10,  1.29479952e+10,  1.29659700e+10,
        3.96187852e+10,  3.96326908e+10,  3.98480859e+10,  3.98622043e+10,
        4.02995954e+10,  4.03138940e+10,  4.08814646e+10,  4.08955035e+10,
        7.98667511e+10,  7.98718302e+10,  8.00466545e+10,  8.00519021e+10,
        8.04098559e+10,  8.04439801e+10,  8.07542992e+10,  8.07882157e+10,
        1.33434766e+11,  1.33443420e+11,  1.33582919e+11,  1.33591436e+11,
        1.33876561e+11,  1.33935533e+11,  1.34120831e+11,  1.34179638e+11,
        2.00344122e+11,  2.00371383e+11,  2.00470089e+11,  2.00497233e+11,
        2.00718724e+11,  2.00807449e+11,  2.00907843e+11,  2.00996431e+11,
        2.80935061e+11,  2.81058409e+11,  2.81088669e+11,  2.81211854e+11,
        2.82804182e+11,  2.82872494e+11,  2.82931978e+11,  2.83000234e+11])

In [70]:
np.diff(D)/1e6

array([1.60160000e+01, 5.47662204e+02, 1.59873304e+01, 1.35009642e+04,
       1.77047268e+01, 2.99124178e+02, 1.79748440e+01, 2.66528152e+04,
       1.39055564e+01, 2.15395145e+02, 1.41183892e+01, 4.37391096e+02,
       1.42986036e+01, 5.67570529e+02, 1.40389616e+01, 3.89712476e+04,
       5.07910788e+00, 1.74824291e+02, 5.24755704e+00, 3.57953800e+02,
       3.41242492e+01, 3.10319041e+02, 3.39165140e+01, 5.26465499e+04,
       8.65460694e+00, 1.39499286e+02, 8.51661696e+00, 2.85124725e+02,
       5.89717953e+01, 1.85298750e+02, 5.88062951e+01, 6.61644847e+04,
       2.72607246e+01, 9.87058515e+01, 2.71445599e+01, 2.21490662e+02,
       8.87244591e+01, 1.00394097e+02, 8.85883837e+01, 7.99386297e+04,
       1.23348491e+02, 3.02599262e+01, 1.23184505e+02, 1.59232875e+03,
       6.83120551e+01, 5.94837576e+01, 6.82560000e+01])

## $\Omega$-splittings
Calculating energy differences between states of opposite parity

state = (J,F1,F)

state1 = (1,1/2,0)

state2 = (1,1/2,1)

state3 = (1,3/2,1)

state4 = (1,3/2,2)

state5 = (2,5/2,2)

state6 = (2,5/2,3)

state7 = (2,3/2,1)

state8 = (2,3/2,2)

In [71]:
#Define the states of interest (approximately)
state1e = 1* CoupledBasisState(F=0, mF = 0, F1 = 1/2, J = 1, I1 = 1/2, I2 = 1/2, P = -1)
state1f = 1* CoupledBasisState(F=0, mF = 0, F1 = 1/2, J = 1, I1 = 1/2, I2 = 1/2, P = +1)

state2e = 1*CoupledBasisState(F=1, mF = 0, F1 = 1/2, J = 1, I1 = 1/2, I2 = 1/2, P = -1)
state2f = 1*CoupledBasisState(F=1, mF = 0, F1 = 1/2, J = 1, I1 = 1/2, I2 = 1/2, P = +1)
                         
state3e = 1*CoupledBasisState(F=1, mF = 0, F1 = 3/2, J = 1, I1 = 1/2, I2 = 1/2, P = -1)
state3f = 1*CoupledBasisState(F=1, mF = 0, F1 = 3/2, J = 1, I1 = 1/2, I2 = 1/2, P = +1)

state4e = 1*CoupledBasisState(F=2, mF = 0, F1 = 3/2, J = 1, I1 = 1/2, I2 = 1/2, P = -1)
state4f = 1*CoupledBasisState(F=2, mF = 0, F1 = 3/2, J = 1, I1 = 1/2, I2 = 1/2, P = +1)

state5e = 1* CoupledBasisState(F=2, mF = 0, F1 = 5/2, J = 2, I1 = 1/2, I2 = 1/2, P = +1)
state5f = 1* CoupledBasisState(F=2, mF = 0, F1 = 5/2, J = 2, I1 = 1/2, I2 = 1/2, P = -1)

state6e = 1*CoupledBasisState(F=3, mF = 0, F1 = 5/2, J = 2, I1 = 1/2, I2 = 1/2, P = +1)
state6f = 1*CoupledBasisState(F=3, mF = 0, F1 = 5/2, J = 2, I1 = 1/2, I2 = 1/2, P = -1)

state7e = 1*CoupledBasisState(F=1, mF = 0, F1 = 3/2, J = 2, I1 = 1/2, I2 = 1/2, P = +1)
state7f = 1*CoupledBasisState(F=1, mF = 0, F1 = 3/2, J = 2, I1 = 1/2, I2 = 1/2, P = -1)

state8e = 1*CoupledBasisState(F=2, mF = 0, F1 = 3/2, J = 2, I1 = 1/2, I2 = 1/2, P = +1)

state8f = 1*CoupledBasisState(F=2, mF = 0, F1 = 3/2, J = 2, I1 = 1/2, I2 = 1/2, P = -1)

E1 = D[find_state_idx_from_state(Hff,state1e,QN)]/1e6 - D[find_state_idx_from_state(Hff,state1f,QN)]/1e6
print(E1)

E2 = D[find_state_idx_from_state(Hff,state2e,QN)]/1e6 - D[find_state_idx_from_state(Hff,state2f,QN)]/1e6
print(E2)

E3 = D[find_state_idx_from_state(Hff,state3e,QN)]/1e6 - D[find_state_idx_from_state(Hff,state3f,QN)]/1e6
print(E3)

E4 = D[find_state_idx_from_state(Hff,state4e,QN)]/1e6 - D[find_state_idx_from_state(Hff,state4f,QN)]/1e6
print(E4)

E5 = D[find_state_idx_from_state(Hff,state5e,QN)]/1e6 - D[find_state_idx_from_state(Hff,state5f,QN)]/1e6
print(E5)

E6 = D[find_state_idx_from_state(Hff,state6e,QN)]/1e6 - D[find_state_idx_from_state(Hff,state6f,QN)]/1e6
print(E6)

E7 = D[find_state_idx_from_state(Hff,state7e,QN)]/1e6 - D[find_state_idx_from_state(Hff,state7f,QN)]/1e6
print(E7)

E8 = D[find_state_idx_from_state(Hff,state8e,QN)]/1e6 - D[find_state_idx_from_state(Hff,state8f,QN)]/1e6
print(E8)

16.016000000000076
15.987330427144116
-17.70472679866907
-17.974843980126025
-13.905556383331714
-14.118389242037665
14.298603628507408
14.038961602193012


In [72]:
find_state_idx_from_state(Hff,state3f,QN)

5

In [73]:
def calculate_splitting(state1,state2):
    return D[find_state_idx_from_state(Hff,state2,QN)]/1e6 - D[find_state_idx_from_state(Hff,state1,QN)]/1e6

## Splittings


In [74]:
ap = calculate_splitting(state1f,state2f)
print("a_+ = {:.2f} MHz".format(ap))

am = calculate_splitting(state1e,state2e)
print("a_- = {:.2f} MHz".format(am))

bp = calculate_splitting(state2f,state3f)
print("b_+ = {:.2f} MHz".format(bp))

bm = calculate_splitting(state2e,state3e)
print("b_- = {:.2f} MHz".format(bm))

cp = calculate_splitting(state3f,state4f)
print("c_+ = {:.2f} MHz".format(cp))

cm = calculate_splitting(state3e,state4e)
print("c_- = {:.2f} MHz".format(cm))

omega1 = calculate_splitting(state2f,state2e)
print("omega1 = {:.2f} MHz".format(omega1))

omega2 = calculate_splitting(state3f,state3e)
print("omega2 = {:.2f} MHz".format(omega2))

a_+ = 563.68 MHz
a_- = 563.65 MHz
b_+ = 13534.66 MHz
b_- = 13500.96 MHz
c_+ = 317.10 MHz
c_- = 316.83 MHz
omega1 = 15.99 MHz
omega2 = -17.70 MHz
